In [1]:
import urllib
import pandas as pd
import numpy as np
from sklearn import datasets

%matplotlib inline
import matplotlib.pyplot as plt
plt.style.use('default')
import xarray as xr
import scipy.interpolate as interp

import scipy.io as sio
import numpy.matlib
import cmath

In [104]:
# Choose iceberg year (2002 - 2015 available)
# Note: Iceberg Season starts in November so many datasets include dates from year-1
season_year = 2015
iip_url_base = 'ftp://sidads.colorado.edu/pub/DATASETS/NOAA/G00807/' 
iip_filename = f'IIP_{season_year}IcebergSeason.csv'
iip_url = iip_url_base + iip_filename
r = urllib.request.urlretrieve(iip_url)
df = pd.read_csv(r[0], converters={'TIME':str})
df['DATETIME'] = pd.to_datetime(df['DATE'] + 'T' + df['TIME'])

In [105]:
# Choose the min number of observations for an eligible iceberg
min_num_obs = 10
eligible_bergs = np.asarray(
    df['BERG_NUMBER'].value_counts()\
    .loc[df['BERG_NUMBER'].value_counts() > min_num_obs].index)

In [106]:
chosen_inds_arr = []

for i in range(eligible_bergs.size):

    berg_id = eligible_bergs[i]
    berg_df = df.loc[df['BERG_NUMBER'] == berg_id]
    
    ind0 = berg_df.index.tolist()[0]
    indf = berg_df.index.tolist()[-1]
    
    max_time_dif = np.timedelta64(24*60*3, 'm')
    
    chosen_inds = []

    for j in range(len(berg_df)-1):

        time_dif = (berg_df.DATETIME.values[j+1] - \
                    berg_df.DATETIME.values[j]).astype('timedelta64[m]')
        
        if time_dif < max_time_dif:
            chosen_inds.append(j+ind0)

        elif len(chosen_inds) > 5:
            chosen_inds_arr.append(chosen_inds)
            chosen_inds = []
        else:
            chosen_inds = []

    if len(chosen_inds) > 5:
        chosen_inds_arr.append(chosen_inds)

In [113]:
# Choose which iceberg track to look at (index)
chosen_track_ind = 2

berg_df = df.loc[chosen_inds_arr[chosen_track_ind]].reset_index()

In [114]:
class Iceberg:
    
    def __init__(self, id_num, times, lats, lons, size):
        self.id_num = id_num
        self.times = times
        self.lats = lats
        self.lons = lons
        if type(size) == str:
            self.length, self.width, self.height = self.get_berg_dims(size)
        elif type(size) == list and len(size) == 3:
            self.length, self.width, self.height = size[0], size[1], size[2]
        else:
            print('Invalid size argument')
            
    def get_berg_dims(self, size):
        # Size must be GR, BB, SM, MED, LG, VLG
        # See https://nsidc.org/data/g00807 for more info
        if size == 'GR':
            l = (0+5)/2; w = (0+5)/2; h = (0+1)/2*10
        elif size == 'BB':
            l = (5+15)/2; w = (5+15)/2; h = (1+5)/2*10        
        elif size == 'SM':
            l = (15+60)/2; w = (15+60)/2; h = (5+15)/2*10        
        elif size == 'MED':
            l = (60+120)/2; w = (60+120)/2; h = (15+45)/2*10               
        elif size == 'LG':
            l = (120)/2; w = (120)/2; h = (45+75)/2*10                
        elif size == 'VLG':
            # Sizes have no listed upper bound
            l = (200+200/2)/2; w = (200+200/2)/2; h = (75+75/2)/2*10     
        # This info for GEN is wrong!
        elif size == 'GEN':
            l = (120)/2; w = (120)/2; h = (45+75)/2*10            
        else:
            print('unknown size class')
            l = None; w = None; h = None
        return l, w, h

In [115]:
iip_berg = Iceberg(berg_df['BERG_NUMBER'].loc[0],
                   berg_df['DATETIME'].loc[0:1].tolist(),
                   berg_df['LATITUDE'].loc[0:1].tolist(),
                   berg_df['LONGITUDE'].loc[0:1].tolist(),
                   berg_df['SIZE'].loc[0])

In [123]:
class GLB:
    
    glb_url = 'http://tds.hycom.org/thredds/dodsC/GLBv0.08/expt_56.3'
    
    def __init__(self, berg):
        self.ds = xr.open_dataset(self.glb_url, 
                                  decode_times=False).sel(
                                  depth=0.0, 
                                  lat = slice(min(berg.lats)-1, max(berg.lats)+1), 
                                  lon = slice(min(berg.lons)-1, max(berg.lons)+1), 
                                  time=slice(self.convert_time(berg.times[0]), 
                                             self.convert_time(berg.times[-1])))
        self.times = np.asarray(self.ds.time)
        self.lats = np.asarray(self.ds.lat)
        self.lons = np.asarray(self.ds.lon)
        #self.water_u = self.ds.water_u
        #self.water_u = np.asarray(self.ds.water_u)
        self.water_u = np.asarray(self.ds.water_u)
        self.water_v = np.asarray(self.ds.water_v)
        self.water_temp = np.asarray(self.ds.water_temp)
        
    def convert_time(self, berg_time):
        date2000 = np.datetime64('2000-01-01')
        diff = berg_time - date2000
        days = diff.days
        seconds = diff.seconds
        hours = days*24 + seconds/3600
        return hours

In [124]:
glb = GLB(iip_berg)

In [142]:
class Navgem:
    navgem_url = 'http://coastwatch.pfeg.noaa.gov/erddap/griddap/erdNavgem05D10mWind_LonPM180'

    def __init__(self, berg):
        self.ds = xr.open_dataset(self.navgem_url).sel(
                         time=slice(self.convert_time(berg.times[0]), 
                                    self.convert_time(berg.times[-1], add_day=True)), 
                         latitude=slice(min(berg.lats)-1, max(berg.lats)+1), 
                         longitude=slice(min(berg.lons)-1, max(berg.lons)+1))
        self.times = np.asarray(self.ds.time)
        self.lats = np.asarray(self.ds.latitude)
        self.lons = np.asarray(self.ds.longitude)
        #self.wind_u = np.asarray(self.ds.wnd_ucmp_height_above_ground)
        #self.wind_v = np.asarray(self.ds.wnd_vcmp_height_above_ground)
                                    
    def convert_time(self, berg_time, add_day=False):
        if add_day:
            one_day = pd.Timedelta('1 days')
            date = '{}-{}-{}' .format((berg_time + one_day).year,
                                      (berg_time + one_day).month,
                                      (berg_time + one_day).day)
        else:
            date = '{}-{}-{}' .format(berg_time.year,
                                      berg_time.month,
                                      berg_time.day)
        return date

In [143]:
navgem = Navgem(iip_berg)

OSError: [Errno -90] NetCDF: file not found: b'http://coastwatch.pfeg.noaa.gov/erddap/griddap/erdNavgem05D10mWind_LonPM180'